In [ ]:
import pandas as pd
import boto3
import json

In [ ]:
# Load DWH Params from a file

import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

CLUSTER_TYPE       = config.get("CLUSTER","CLUSTER_TYPE")
NUM_NODES          = config.get("CLUSTER","NUM_NODES")
NODE_TYPE          = config.get("CLUSTER","NODE_TYPE")

CLUSTER_IDENTIFIER = config.get("CLUSTER","CLUSTER_IDENTIFIER")
DB_NAME            = config.get("CLUSTER","DB_NAME")
DB_USER            = config.get("CLUSTER","DB_USER")
DB_PASSWORD        = config.get("CLUSTER","DB_PASSWORD")
PORT               = config.get("CLUSTER","DB_PORT")
REGION             = config.get("CLUSTER","REGION")
IAM_ROLE_NAME      = config.get("CLUSTER","IAM_ROLE_NAME")
ARN                = config.get("IAM_ROLE", "ARN")

(DB_USER, DB_PASSWORD, DB_NAME)

pd.DataFrame({"Param":
                  ["CLUSTER_TYPE", "NUM_NODES", "DNODE_TYPE", "CLUSTER_IDENTIFIER", "DB_NAME", "DB_USER", "DB_PASSWORD", "PORT", "IAM_ROLE_NAME", "REGION", "ARN"],
              "Value":
                  [CLUSTER_TYPE, NUM_NODES, NODE_TYPE, CLUSTER_IDENTIFIER, DB_NAME, DB_USER, DB_PASSWORD, PORT, IAM_ROLE_NAME, REGION, ARN]
             })

In [ ]:
# Create clients for EC2, S3, IAM, and Redshift

# s3 = boto3.resource('s3',
#                        region_name = REGION,
#                        aws_access_key_id = KEY,
#                        aws_secret_access_key = SECRET
#                    )

iam = boto3.client('iam',aws_access_key_id = KEY,
                     aws_secret_access_key = SECRET,
                     region_name = REGION
                  )

redshift = boto3.client('redshift',
                       region_name = REGION,
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

In [ ]:
# Create an IAM Role that makes Redshift able to access S3 bucket (ReadOnly)

try:
    print('1.1 Creating a new IAM Role')
    dwhRole = iam.create_role(
        Path='/',
        RoleName=IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    
except Exception as e:
    print(e)
    dwhRole = iam.get_role(RoleName=IAM_ROLE_NAME)

In [ ]:
#  Attaching Policy

print('1.2 Attaching Policy')
iam.attach_role_policy(RoleName=IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

In [ ]:
# Get the Role ARN

print('1.3 Get the IAM role ARN')
roleArn = iam.get_role(RoleName=IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

In [ ]:
# Create a RedShift Cluster

try:
    response = redshift.create_cluster(        
        #HW
        ClusterType = CLUSTER_TYPE,
        NodeType = NODE_TYPE,
        NumberOfNodes = int(NUM_NODES),

        #Identifiers & Credentials
        DBName = DB_NAME,
        ClusterIdentifier = CLUSTER_IDENTIFIER,
        MasterUsername = DB_USER,
        MasterUserPassword = DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles = [roleArn]
    )
except Exception as e:
    print(e)

In [ ]:
# Describe the cluster to see its status

def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

In [ ]:
# Cluster status is 'available'?

redshift.describe_clusters(ClusterIdentifier=CLUSTER_IDENTIFIER)['Clusters'][0]["ClusterStatus"]

In [ ]:
# Take note of the cluster endpoint and role ARN

ENDPOINT = myClusterProps['Endpoint']['Address']
IAM_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("ENDPOINT :: ", ENDPOINT)
print("IAM_ROLE_ARN :: ", IAM_ROLE_ARN)

In [ ]:
# Make sure you can connect to the clusterConnect to the cluster and run a few queries

%load_ext sql

conn_string = f"postgresql://{DB_USER}:{DB_PASSWORD}@{ENDPOINT}:{PORT}/{DB_NAME}"
print(conn_string)
%sql $conn_string

In [ ]:
# Execute ETL on cluster
%run etl.py

In [ ]:
# Create all tables
%run create_tables.py

In [ ]:
# Number of rows in events staging table

cur.execute("SELECT COUNT(*) FROM events_staging")
cur.fetchall()

In [ ]:
# Number of rows in songs staging table

cur.execute("SELECT COUNT(*) FROM songs_staging")
cur.fetchall()

In [ ]:
# Delete the created resources
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)

In [ ]:
# Delete the created resources

iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)